In [23]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [24]:
import sys
import glob
import pandas as pd
import os
import seaborn as sns

from tqdm import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
from pathlib import Path
import scipy.stats
import time

from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.examples.ubc_plotting_utils import *
from open_spiel.python.examples.ubc_utils import *
import open_spiel.python.examples.ubc_dispatch as dispatch

from auctions.webutils import *

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

from open_spiel.python.examples.ubc_cma import *

from open_spiel.python.games.clock_auction_base import InformationPolicy, ActivityPolicy, UndersellPolicy, TiebreakingPolicy
from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.examples.ubc_decorators import TakeSingleActionDecorator, TremblingAgentDecorator, ModalAgentDecorator

plt.style.use('https://raw.githubusercontent.com/gregdeon/plots/main/style.mplstyle')
plt.matplotlib.rcParams['figure.dpi'] = 300
plt.matplotlib.rcParams['font.size'] = 6


In [25]:
# Load our runs from the "interesting" game
experiments = [
    # 'jan20_story'
    'jan20_boring'
]

runs = []
for experiment in experiments:
    runs += Experiment.objects.get(name=experiment).equilibriumsolverrun_set.all()
print(f"Found {len(runs)} runs")

Found 100 runs


# Analyze runs

In [ ]:
records = []
for run in tqdm(runs):    
    game = run.game.load_as_spiel()
    record = {
        'run_name': run.name,
        'game_name': run.game.name, 
        'seed': run.config.get('seed'), 
        'config': run.get_config_name(),
        'alg': get_algorithm_from_run(run),
    }
    
    record.update(get_game_info(game, run.game))  
    
    record['no_error'] = False
    records.append(record) # Put it here so you see the False's in the display
        
    try:
        game, final_checkpoint, policy = get_results(run, load_policy=record['rho'] > 0)
    except Exception as e:
        print(f"Skipping run {run.name} because of error {e}")
        continue
    
    try:
        
        # # TODO: In general, remvoe this if
        # if not (record['rho'] > 0 and record['config'] == 'cfr_port_10_extexternal_plus_linear' and record['n_types'] == 4):
        #     continue
        
        record['t'] = final_checkpoint.t
        record['walltime'] = run.walltime(),
        evaluation = final_checkpoint.get_modal_eval()
        
        record['nash_conv'] = evaluation.nash_conv
        record['rewards'] = evaluation.mean_rewards
        record['nash_conv_frac'] = evaluation.nash_conv / sum(evaluation.mean_rewards) if not pd.isnull(evaluation.nash_conv) else np.nan
        record['heuristic_conv'] = evaluation.heuristic_conv
        record['heuristic_conv_frac'] = evaluation.heuristic_conv / sum(evaluation.mean_rewards) if not pd.isnull(evaluation.heuristic_conv) else np.nan

        for i in range(game.num_players()):
            record[f'rewards_{i}'] = evaluation.mean_rewards[i]
            record[f'nc_player_improvements_{i}'] = evaluation.nash_conv_player_improvements[i] if not pd.isnull(evaluation.nash_conv) else np.nan
            record[f'nc_player_improvements_frac_{i}'] = (evaluation.nash_conv_player_improvements[i] / evaluation.mean_rewards[i]) if not pd.isnull(evaluation.nash_conv) else np.nan

            record[f'hc_player_improvements_{i}'] = evaluation.heuristic_conv_player_improvements[i] if not pd.isnull(evaluation.heuristic_conv) else np.nan
            record[f'hc_player_improvements_frac_{i}'] = (evaluation.heuristic_conv_player_improvements[i] / evaluation.mean_rewards[i]) if not pd.isnull(evaluation.heuristic_conv) else np.nan

        record.update(**analyze_samples(evaluation.samples, game))

        nc = record['nash_conv']
        hc = record['heuristic_conv']
        # print(f"NashConv = {(np.nan if pd.isnull(nc) else nc):.2f}; HeuristicConv = {(np.nan if pd.isnull(hc) else hc):.2f}")
        
        
        ### Parse the straightforward story for the game
        straightforward_eval = final_checkpoint.get_straightforward_eval()
        straightforward_metrics = analyze_samples(straightforward_eval.samples, game)
        record.update({f'straightforward_{k}' : v for k, v in straightforward_metrics.items()})
        ### End
        
        # This is slow. Do you really need to do it for both the rho=0 and rho=1 versions of the game? That's a 2X speedup(!)
        if record['rho'] == 0:
            record['straightforward_nash_conv'] = get_straightforward_nash_conv(game)       
        else:
            record['straightforward_nash_conv'] = None
           
        # Commenting out for mem reasons
        if record['rho'] > 0 and record['config'] == 'cfr_port_10_extexternal_plus_linear' and record['n_types'] == 4:
            # What happens in the rho = 0 game?
            record['rho_0_nash_conv'] = get_modal_nash_conv_new_rho(run.game.load_as_spiel(), policy, dict(final_checkpoint.equilibrium_solver_run.config), rho=0)
        
        record['no_error'] = True
    except Exception as e:
        print(f"Something wrong with {run}. Skipping. {e}")
        # raise e
        # break
        import traceback
        print(traceback.format_exc())

print(len(records))

 28%|███████████████████████████████████████████████▉                                                                                                                           | 28/100 [40:57<4:14:58, 212.47s/it]

In [7]:
df.query(
    """ rho > 0 and config == 'cfr_port_10_extexternal_plus_linear' and n_types == 4
    """
)['rho_0_nash_conv'].describe()

count    50.000000
mean      0.004131
std       0.016519
min       0.000000
25%       0.000000
50%       0.000000
75%       0.000000
max       0.068857
Name: rho_0_nash_conv, dtype: float64

In [5]:
df = pd.DataFrame.from_records(records)
with pd.option_context('display.max_rows', None):
    display(df[['value_structure', 'rule', 'base_game_name', 'deviations', 'no_error']].value_counts().sort_index())

value_structure  rule       base_game_name  deviations  no_error
quasi_linear     base       jan19_0_base    1000        False       20
                            jan19_10_base   1000        False       20
                            jan19_11_base   1000        False       20
                            jan19_12_base   1000        False       20
                            jan19_13_base   1000        False       20
                            jan19_14_base   1000        False       20
                            jan19_15_base   1000        False       15
                                                        True         5
                            jan19_16_base   1000        False       15
                                                        True         5
                            jan19_17_base   1000        False       15
                                                        True         5
                            jan19_18_base   1000        False       15
            

In [10]:
# df.to_csv('cached_results_jan19.csv', index=False)
df = pd.read_csv('cached_results_jan19.csv')

In [20]:
df4t = df.query('n_types == 4 and rho == 1 and config == "cfr_port_10_extexternal_plus_linear"')
pd.concat((df4t['rewards_0'], df4t['rewards_1'])).describe()

count    100.000000
mean      14.316057
std        3.868249
min        6.335326
25%       12.607385
50%       14.109617
75%       17.505699
max       20.023663
dtype: float64

In [ ]:
# df['straightforward_nash_conv'].plot(kind='hist')

In [ ]:
df['nash_conv'].plot(kind='hist')
print(df['nash_conv'].isnull().sum())

df.query('nash_conv.isnull()')[['value_structure', 'rule', 'base_game_name', 'deviations', 'no_error']].value_counts().sort_index()
# Huh???? Why are 0 deviation games showing as null. Is that for real?

In [ ]:
palette = dict()
colors = ['red', 'blue', 'magenta', 'green', 'orange', 'brown', 'black', 'navy', 'pink', 'gold', 'darkgreen', 'orangered', 'olive']
# for i, v in enumerate(df['variant'].unique()):
#     palette[v] = colors[i]

In [ ]:
# Need to a) Remove "bad" entries b) Be careful about comparisons that are missing datapoints 
df_plt = df.copy()

In [ ]:
### Remove bad entries

good_thresh = 0.1
df_plt = df.query(f'hc_player_improvements_frac_0 < {good_thresh} and hc_player_improvements_frac_1 < {good_thresh}').copy()
# df_plt = df.query(f'nash_conv_frac < {good_thresh}')
len(df), len(df_plt)

In [ ]:


# 1) Get max/min for each valuation/treatment pairing over each stat

# First query down to relevant datapoints. Then groupby rule change and SATS =(game_name) and max/min?
metrics = ['total_revenue', 'total_welfare', 'auction_lengths', 'num_lotteries', 'p_lottery', 'exposure_frac']
for i in range(2): # TODO:
    metrics += [f'p{i}_utility', f'p{i}_payment']


df_plt_indexed = df_plt.set_index(['value_structure', 'rule', 'deviations']).sort_index().copy()

def make_data_dict(df):
    data = dict()
    for metric in metrics:
        data[f'max_{metric}'] = df.groupby('base_game_name')[metric].max()
        data[f'min_{metric}'] = df.groupby('base_game_name')[metric].min()
    return pd.DataFrame(data)
    
for idx, grp_df in df_plt.groupby(['value_structure', 'rule', 'deviations']):
    if idx[1] == 'base':
        continue
        
    try:
        data_grp_df = make_data_dict(grp_df)
        normalizer_grp_df = df_plt_indexed.loc[(idx[0], 'base')]
        data_normalized_df = make_data_dict(normalizer_grp_df)

        cmap_norm = plt.matplotlib.colors.TwoSlopeNorm(vmin=0.9, vcenter=1, vmax=1.1)
        cmap = plt.cm.get_cmap('RdBu').copy()
        cmap.set_bad('magenta')
        plt.figure(figsize=(12, 8))
        data = (data_grp_df / data_normalized_df).values.T
        plt.imshow(data, cmap=cmap, norm=cmap_norm)
        plt.title(idx)
        plt.yticks(range(len(data_grp_df.columns)), data_grp_df.columns)
        plt.colorbar()
        plt.show()
    except Exception as e:
        print(idx, e)

In [ ]:

df['rho'].value_counts()

In [ ]:
df['config'].value_counts()

In [ ]:
df_plt = df.query('rho == 0 and n_types == 1 and config == "cfr_port_10_extexternal_plus_linear"')
print(len(df_plt))

# for metric_name in ['straightforward_auction_lengths', 'straightforward_num_lotteries', 'straightforward_unsold_licenses', 'straightforward_total_revenue', 'straightforward_total_welfare']:
for metric_name in ['auction_lengths', 'num_lotteries', 'unsold_licenses', 'total_revenue', 'total_welfare']:
    plt.figure(figsize=(12, 4))
    sns.swarmplot(data=df_plt, x='base_game_name', y=metric_name, hue='tiebreaking_policy', dodge=True, size=4)
    # plt.gca(); plt.legend([], [], frameon=False)
    plt.title(metric_name)
    plt.xticks(rotation=90)
    plt.show()

In [22]:
#### When is nash conv especially high if you just bid straightforwardly?

# df.query('rho == 0 and config == "cfr_port_10_extexternal_plus_linear"').groupby(['n_types', 'tiebreaking_policy'])['straightforward_nash_conv'].max().sort_values()
df.query('rho == 0 and config == "cfr_port_10_extexternal_plus_linear"').groupby(['n_types', 'tiebreaking_policy'])['straightforward_nash_conv'].mean().sort_values()

n_types  tiebreaking_policy
1        DROP_BY_LICENSE       0.000000
         DROP_BY_PLAYER        0.000000
4        DROP_BY_PLAYER        0.136661
3        DROP_BY_PLAYER        0.343663
2        DROP_BY_PLAYER        0.668527
3        DROP_BY_LICENSE       1.230711
4        DROP_BY_LICENSE       1.837529
2        DROP_BY_LICENSE       2.032346
Name: straightforward_nash_conv, dtype: float64

In [ ]:
df['nash_conv'] # Would need to compute

In [ ]:
df_plt['nash_conv'].describe()

In [ ]:
# # What is the graphical straightforward story?


# straightforward_df = df.query('rho == 0 and config == "cfr_port_10_extexternal_plus_linear" and seed == 100')
# # Group by num types

# g = sns.FacetGrid(straightforward_df, row="n_types", sharex=False)
# g.map_dataframe(sns.scatterplot, x='straightforward_total_welfare', y='straightforward_total_revenue', hue='tiebreaking_policy', style='n_types')


# # sns.scatterplot(data=straightforward_df, x='straightforward_total_welfare', y='straightforward_total_revenue', hue='tiebreaking_policy', style='n_types')


In [ ]:
# strategy_df = df.query('rho == 1 and config == "cfr_port_10_extexternal_plus_linear"')
# print(len(strategy_df))
# # 5 games * 5 seeds * 2 tiebreaks * 4 types

# # Group by num types
# # Trouble: can't see overlapping

# g = sns.FacetGrid(strategy_df, row="n_types", sharex=False)
# g.map_dataframe(sns.scatterplot, x='total_welfare', y='total_revenue', hue='tiebreaking_policy', style='tiebreaking_policy')
# g.add_legend()


In [ ]:
def make_plots(df, straightforward=False):
    sample_idx = df.groupby('n_types')['base_game_name'].unique()
    df['sample_id'] = df.apply(lambda row: sample_idx[row['n_types']].tolist().index(row['base_game_name']), axis=1)
    # Group by num types

    metrics = ['total_revenue', 'total_welfare', 'auction_lengths', 'num_lotteries']
    if straightforward:
        metrics = [f'straightforward_{m}' for m in metrics]

    # Dropping duplicates reduces visual noise. What I'd prefer to do though is drop any row entirely if it isn't an extremal point of it's sample_id for at least one metric
    df = df[
        metrics + ['tiebreaking_policy', 'n_types', 'sample_id']
    ].round(1).drop_duplicates()


    df = df.melt(
        id_vars=['n_types', 'sample_id', 'tiebreaking_policy'],
        value_vars=metrics,
        var_name='metric',
    )

    sns.catplot(
        data=df, kind="swarm",
        col="n_types",
        row='metric',
        x="sample_id", y="value", hue="tiebreaking_policy", 
        aspect=.5,
        sharey='row'
    )

make_plots(df.query('rho == 0 and config == "cfr_port_10_extexternal_plus_linear" and n_types > 1').copy(), straightforward=True)

In [ ]:
make_plots(df.query('rho == 1 and config == "cfr_port_10_extexternal_plus_linear"').copy())

In [ ]:
make_plots(df.query('rho == 1 and config == "cfr_port_10_extexternal_plus_linear_no_trem"').copy())

In [9]:
df.to_csv('jan19_story_with_rho0_ncs_where_relevant_but_missing_other_stuff.csv', index=False)